# Create ocean mask

A very straighforward notebook to generate ocean mask for ice-shelves and no ice-shelves configurations.

In [1]:
import xarray as xr
import numpy as np

In [2]:
hgrid = xr.open_dataset('ocean_hgrid.nc')

In [3]:
OM4 = xr.Dataset()
lon = hgrid['x'].values[1::2,1::2].copy()
lat = hgrid['y'].values[1::2,1::2].copy()
OM4['lon'] = xr.DataArray(data=lon, dims=('yh', 'xh'), attrs={'units': 'degrees_east'})
OM4['lat'] = xr.DataArray(data=lat, dims=('yh', 'xh'), attrs={'units': 'degrees_north'})

## without antarctic iceshelves

In [4]:
topog = xr.open_dataset('ocean_topog_noiceshelves.nc')

In [5]:
depth = xr.DataArray(data=topog['depth'].values, dims=('yh', 'xh'))
OM4['mask'] = xr.where(depth >0, 1, 0).astype(np.float64)
OM4['mask'].attrs = {"standard_name": "ocean fraction at T-cell centers",
                     "units": "none"}
OM4['mask'].encoding = {"_FillValue": -1.0e+20}

mask = OM4.drop_vars(['lon', 'lat']).rename({'yh': 'ny', 'xh': 'nx'})
mask.to_netcdf('ocean_mask_noiceshelves.nc',
               format='NETCDF3_64BIT', engine='netcdf4')

## with antarctic iceshelves

In [6]:
topog = xr.open_dataset('ocean_topog.nc')

In [7]:
depth = xr.DataArray(data=topog['depth'].values, dims=('yh', 'xh'))
OM4['mask'] = xr.where(depth >0, 1, 0).astype(np.float64)
OM4['mask'].attrs = {"standard_name": "ocean fraction at T-cell centers",
                     "units": "none"}
OM4['mask'].encoding = {"_FillValue": -1.0e+20}

mask = OM4.drop_vars(['lon', 'lat']).rename({'yh': 'ny', 'xh': 'nx'})
mask.to_netcdf('ocean_mask_iceshelves.nc',
               format='NETCDF3_64BIT', engine='netcdf4')